In [1]:
import numpy as np
from tqdm.auto import tqdm
import scipy.sparse as sps
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from numba import njit, prange

In [3]:
from recommenders.mfi.utils import build_conf_mat, build_pref_mat

In [4]:
train = sps.load_npz('./data/train1.npz')
test = sps.load_npz('./data/test1.npz')

In [5]:
n_users, n_items = train.shape
Cui_sparse = build_conf_mat(train)
Pui_sparse = build_pref_mat(train)

In [6]:
print(n_users, n_items)

148438 89276


In [7]:
K = 100

In [8]:
user_factors = np.random.normal(scale=1./K, size=(n_users, K))
item_factors = np.random.normal(scale=1./K, size=(n_items, K))

In [9]:
@njit
def nonzeros(u, rows, cols, vals):
    indices = np.nonzero(rows == u)
    return cols[indices], vals[indices]

In [10]:
%%timeit
nonzeros(2, Cui_sparse.row, Cui_sparse.col, Cui_sparse.data)

3.83 ms ± 59.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
@njit
def build_lin_equation(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors):
    A = YtY + regularization * np.eye(n_factors)
    b = np.zeros(n_factors)
    
    nz_rows, nz_vals = nonzeros(u, Cui_rows, Cui_cols, Cui_vals)
    for _i in range(nz_rows.size):
        i = nz_rows[_i]
        confidence = nz_vals[_i]
        factor = Y[i]

        # need to make change here I think wrt. preferences
        if confidence > 0:
            b += confidence * factor
        else:
            confidence *= -1

        A += (confidence - 1) * np.outer(factor, factor)
    return A, b

In [12]:
Y = item_factors
YtY = item_factors.T.dot(item_factors)

In [13]:
%%timeit
build_lin_equation(Y, YtY, 2, Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, 0.01, 100)

4.15 ms ± 54.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
@njit
def user_factor(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors):
    # Xu = (YtCuY + regularization * I)^-1 (YtCuPu)
    A, b = build_lin_equation(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors)
    return np.linalg.solve(A, b)

In [15]:
%%timeit
user_factor(Y, YtY, 2, Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, 0.01, 100)

6.47 ms ± 164 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
def least_squares(Cui_rows, Cui_cols, Cui_vals, X, Y, regularization):
    
    users, n_factors = X.shape
    YtY = Y.T.dot(Y)

    for u in tqdm(range(users)):
        X[u] = user_factor(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors)

In [17]:
%%time
least_squares(Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, user_factors, item_factors, 0.01)

  0%|          | 0/148438 [00:00<?, ?it/s]

Wall time: 17min 21s


In [18]:
user_factors = np.random.normal(scale=1./K, size=(n_users, K))
item_factors = np.random.normal(scale=1./K, size=(n_items, K))

In [24]:
@njit
def njit_least_squares(Cui_rows, Cui_cols, Cui_vals, X, Y, regularization):
    
    users, n_factors = X.shape
    YtY = Y.T.dot(Y)

    for u in range(users):
        X[u] = user_factor(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors)

In [25]:
%%time
njit_least_squares(Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, user_factors, item_factors, 0.01)

Wall time: 17min 4s


In [21]:
user_factors = np.random.normal(scale=1./K, size=(n_users, K))
item_factors = np.random.normal(scale=1./K, size=(n_items, K))

In [22]:
@njit(parallel=True)
def njitp_least_squares(Cui_rows, Cui_cols, Cui_vals, X, Y, regularization):
    
    users, n_factors = X.shape
    YtY = Y.T.dot(Y)

    for u in prange(users):
        X[u] = user_factor(Y, YtY, u, Cui_rows, Cui_cols, Cui_vals, regularization, n_factors)

In [23]:
%%time
njitp_least_squares(Cui_sparse.row, Cui_sparse.col, Cui_sparse.data, user_factors, item_factors, 0.01)

Wall time: 10min 7s
